In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/fleurs-my-ms/resolve/main/test-fleurs.json
# !wget https://huggingface.co/datasets/malaysia-ai/fleurs-my-ms/resolve/main/fleurs-test.zip
# !unzip fleurs-test.zip

In [2]:
from glob import glob
import json
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from datasets import Audio
from tqdm import tqdm
import jiwer

sr = 16000
audio = Audio(sampling_rate=sr)
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [3]:
processor = AutoProcessor.from_pretrained('mesolitica/malaysian-whisper-tiny')
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    'mesolitica/malaysian-whisper-tiny',
    use_flash_attention_2 = True,
    torch_dtype = torch.bfloat16
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
_ = model.cuda()

In [5]:
with open('test-fleurs.json') as fopen:
    data = json.load(fopen)

In [6]:
len(data)

749

In [7]:
wer, cer = [], []

for i in tqdm(range(len(X))):
    
    
    y, _ = malaya_speech.load(X[i])
    actual = Y[i]
    
    inputs = processor([y], return_tensors = 'pt', sampling_rate = 16000)
    inputs['input_features'] = inputs['input_features'].type(torch.bfloat16).cuda()
    r = model.generate(inputs['input_features'], language='ms', return_timestamps=True)
    out = processor.tokenizer.decode(r[0], skip_special_tokens = True).strip()
    out = preprocessing_text(out)
        
    actual = actual.lower()
    out = out.lower()
    
    wer_ = calculate_wer(actual, out)
    if wer_ > 1:
        wer_ = 1.0
    
    cer_ = calculate_cer(actual, out)
    if cer_ > 1:
        cer_ = 1.0
    
    wer.append(wer_)
    cer.append(cer_)

100%|██████████| 749/749 [05:06<00:00,  2.44it/s]


In [8]:
import numpy as np

np.mean(wer), np.mean(cer)

(0.3461808122686204, 0.13390519685940314)